Import Libraries

In [ ]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, f1_score, precision_score, recall_score, classification_report

Load Data

In [3]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step


Data Shaping

In [5]:
x_train = x_train.reshape(-1, 28*28).astype('float32') / 255.0
x_test = x_test.reshape(-1, 28*28).astype('float32') / 255.0
y_train = y_train.astype('int32')
y_test = y_test.astype('int32')

Print shape

In [41]:
print(f'Train Images Shape: {x_train.shape}')
print(f'Train Labels Shape: {y_train.shape}')
print(f'Test Images Shape: {x_test.shape}')
print(f'Test Labels Shape: {y_test.shape}')

Train Images Shape: (60000, 784)
Train Labels Shape: (60000,)
Test Images Shape: (10000, 784)
Test Labels Shape: (10000,)
